In [27]:
import numpy as np
import pandas as pd
import os
import csv
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from scipy.stats import gmean

In [28]:
folder_path = '/home/ximo/Escritorio/ProyectoTFG/featuresExtended'
output_path = '/home/ximo/Escritorio/ProyectoTFG/results'
fichero_con_resultados = '/home/ximo/Escritorio/ProyectoTFG/results/RESULTS.csv'
report_file_path = '/home/ximo/Escritorio/ProyectoTFG/results/RESULTS.txt'

def calc_metrics(cm, labels):
    """Calcula las métricas basadas en la matriz de confusión para cada clase y en general."""
    # Verifica que la matriz de confusión sea cuadrada
    assert cm.shape[0] == cm.shape[1], "La matriz de confusión debe ser cuadrada."
    
    # Accuracy general
    acc = np.trace(cm) / np.sum(cm)  
    
    # Sensibilidad (recall) por clase
    sens = cm.diagonal() / cm.sum(axis=1)  
    
    # Predicciones y etiquetas verdaderas concatenadas
    true_labels = np.concatenate([np.full(fill_value=label, shape=(int(cm[i, :].sum()),), dtype=int) for i, label in enumerate(labels)])
    pred_labels = np.concatenate([np.full(fill_value=label, shape=(int(cm[:, j].sum()),), dtype=int) for j, label in enumerate(labels)])
        
    # F1-Score por clase
    f1_scores = f1_score(true_labels, pred_labels, average=None)
    
    # G-Mean general (geometric mean de las sensibilidades)
    gmean_value = gmean(sens)
    
    # Cálculo correcto de especificidad por clase
    spec = []
    for i in range(len(labels)):
        tn = np.sum(cm) - (np.sum(cm[:, i]) + np.sum(cm[i, :]) - cm[i, i])
        fp = np.sum(cm[:, i]) - cm[i, i]
        spec.append(tn / (tn + fp))
    spec = np.array(spec)
    
    # Sensibilidad general (promedio de las sensibilidades por clase)
    sens_general = np.mean(sens)
    
    # Especificidad general (promedio de las especificidades por clase)
    spec_general = np.mean(spec)
    
    # F1-Score general (promedio de los F1-Scores por clase)
    f1_general = np.mean(f1_scores)
    
    return {
        "Accuracy General": acc,
        "G-Mean General": gmean_value,
        "F1-Score General": f1_general,
        "Sensibilidad Promedio": sens_general,
        "Especificidad General": spec_general,
        "F1-Scores por Clase": f1_scores,
        "Sensibilidades por Clase": sens,
        "Especificidades por Clase": spec
    }

In [29]:

def summarize_results(output_path):
    results = []
    activity_labels = {0: 'Música', 1: 'Leer', 2: 'Meditar', 3: 'Matemáticas'}
    columns = ['Model', 'Window', 'K', 'Accuracy', 'G-Mean', 'F1-Score General', 'Sensibilidad Promedio', 'Especificidad General']

    # Leer el archivo de tiempos de modelo
    time_df_path = os.path.join(output_path, "model_times.csv")
    if os.path.exists(time_df_path):
        time_df = pd.read_csv(time_df_path)
        time_df['Window'] = time_df['Window'].astype(str)
        time_df['K'] = time_df['K'].astype(str)
    else:
        print("Archivo de tiempos no encontrado.")
        return pd.DataFrame(columns=columns)  # Devolver DataFrame vacío si no se encuentra el archivo

    first_file = True
    for file_name in os.listdir(output_path):
        if file_name.startswith("conf_matrix_"):
            parts = file_name.split('_')
            window = parts[2]
            model = parts[3]
            k = parts[4].replace('.csv', '')

            cm_df = pd.read_csv(os.path.join(output_path, file_name), index_col=0)
            cm = cm_df.values
            labels = [int(label) for label in cm_df.index]

            # Configurar columnas dinámicas en la primera iteración
            if first_file:
                for label in labels:
                    label_str = activity_labels[label]
                    columns.extend([f'F1-Score_{label_str}', f'Sensitivity_{label_str}', f'Specificity_{label_str}'])
                columns.append('Time(s)')  # Agregar la columna de tiempo promedio al final
                first_file = False

            metrics = calc_metrics(cm, labels)
            row = [
                model, window, k, metrics["Accuracy General"], 
                metrics["G-Mean General"], metrics["F1-Score General"], metrics["Sensibilidad Promedio"], metrics["Especificidad General"]
            ]
            for f1, s, sp in zip(metrics["F1-Scores por Clase"], metrics["Sensibilidades por Clase"], metrics["Especificidades por Clase"]):
                row.extend([f1, s, sp])

            # Obtener el tiempo de ejecución del modelo
            time_row = time_df[(time_df['Model'] == model) & (time_df['Window'] == window) & (time_df['K'] == k)]
            if not time_row.empty:
                model_time = time_row['Time(s)'].iloc[0]
            else:
                model_time = 'N/A'
            row.append(model_time)

            results.append(row)

    results_df = pd.DataFrame(results, columns=columns)
    return results_df

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)
results_df.to_csv(fichero_con_resultados, index=False)  # Ajusta esta ruta según sea necesario


In [4]:
import os
import pandas as pd

def load_results(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        df_list.append(df)
    results_df = pd.concat(df_list, ignore_index=True)
    drop_cols = [col for col in results_df.columns if any(char.isdigit() for char in col)]
    results_df.drop(columns=drop_cols, inplace=True)
    return results_df

def find_best_metrics(results_df):
    max_accuracy = results_df['Accuracy'].max()
    max_bal_acc = results_df['Sensibilidad Promedio'].max()
    max_g_mean = results_df['G-Mean'].max()

    best_accuracy = results_df[results_df['Accuracy'] == max_accuracy]
    best_bal_acc = results_df[results_df['Sensibilidad Promedio'] == max_bal_acc]
    best_g_mean = results_df[results_df['G-Mean'] == max_g_mean]

    return best_accuracy, best_bal_acc, best_g_mean

def format_confusion_matrix(folder_path, model, window, k):
    k = int(k)
    window = int(window)
    filename = f"conf_matrix_{window}_{model}_{k}.csv"
    file_path = os.path.join(folder_path, filename)
    cm_df = pd.read_csv(file_path, index_col=0)
    activity_labels = {0: 'Música', 1: 'Leer', 2: 'Meditar', 3: 'Matemáticas'}
    cm_df.columns = cm_df.columns.astype(int)
    cm_df.index = cm_df.index.map(activity_labels)
    cm_df.columns = cm_df.columns.map(activity_labels)
    cm_normalized = cm_df.div(cm_df.sum(axis=1), axis=0)
    return cm_normalized.round(3).to_string()

def create_report_file(folder_path, results_df, output_file_path):
    best_accuracy, best_bal_acc, best_g_mean = find_best_metrics(results_df)
    
    with open(output_file_path, 'w') as file:
        file.write("Best Accuracy Models Info:\n")
        file.write(best_accuracy.to_string())
        file.write("\n\nMatrices de Confusión Normalizadas por Filas (Accuracy):\n")
        for _, row in best_accuracy.iterrows():
            file.write(f"Model: {row['Model']} - Window: {row['Window']} - K: {row['K']}\n")
            file.write(format_confusion_matrix(folder_path, row['Model'], row['Window'], row['K']))
            file.write("\n")

        file.write("\nBest mean Recall Models Info:\n")
        file.write(best_bal_acc.to_string())
        file.write("\n\nMatrices de Confusión Normalizadas por Filas (mean recall`):\n")
        for _, row in best_bal_acc.iterrows():
            file.write(f"Model: {row['Model']} - Window: {row['Window']} - K: {row['K']}\n")
            file.write(format_confusion_matrix(folder_path, row['Model'], row['Window'], row['K']))
            file.write("\n")

        file.write("\nBest G-Mean Models Info:\n")
        file.write(best_g_mean.to_string())
        file.write("\n\nMatrices de Confusión Normalizadas por Filas (G-Mean):\n")
        for _, row in best_g_mean.iterrows():
            file.write(f"Model: {row['Model']} - Window: {row['Window']} - K: {row['K']}\n")
            file.write(format_confusion_matrix(folder_path, row['Model'], row['Window'], row['K']))
            file.write("\n")

    print(f'Report has been saved to {output_file_path}')

results_df = load_results(output_path)
create_report_file(output_path, results_df, report_file_path)


Report has been saved to /home/ximo/Escritorio/ProyectoTFG/results/RESULTS.txt


In [9]:
def load_results(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        df_list.append(df)
    results_df = pd.concat(df_list, ignore_index=True)
    drop_cols = [col for col in results_df.columns if any(char.isdigit() for char in col)]
    results_df.drop(columns=drop_cols, inplace=True)
    # Asegurar que NaN en la columna 'Model' no cause problemas
    results_df = results_df[results_df['Model'].str.contains('StackingClassifier', na=False)]
    return results_df

def find_best_by_metric(results_df, metric):
    results_grouped = results_df.groupby('Window')
    best_models = {}
    for window, group in results_grouped:
        max_value = group[metric].max()
        best_models[window] = group[group[metric] == max_value]
    return best_models

def create_stacking_report(folder_path, output_file_path):
    results_df = load_results(folder_path)
    
    best_by_accuracy = find_best_by_metric(results_df, 'Accuracy')
    best_by_bal_acc = find_best_by_metric(results_df, 'Sensibilidad Promedio')
    best_by_g_mean = find_best_by_metric(results_df, 'G-Mean')

    with open(output_file_path, 'w') as file:
        file.write("StackingClassifier Performance Report\n\n")
        
        windows = results_df['Window'].unique()
        for window in sorted(windows):
            file.write(f"Window: {window}\n")
            if window in best_by_accuracy:
                file.write("Best Accuracy:\n")
                file.write(best_by_accuracy[window].to_string(index=False))
                file.write("\n\n")
            if window in best_by_bal_acc:
                file.write("Best mean Recall:\n")
                file.write(best_by_bal_acc[window].to_string(index=False))
                file.write("\n\n")
            if window in best_by_g_mean:
                file.write("Best G-Mean:\n")
                file.write(best_by_g_mean[window].to_string(index=False))
                file.write("\n\n")
    
    print(f'Report has been saved to {output_file_path}')

# Asegúrate de definir 'output_path' correctamente
report_file_path = '/home/ximo/Escritorio/ProyectoTFG/results/RESULTSSTACKING.txt'
create_stacking_report(output_path, report_file_path)


Report has been saved to /home/ximo/Escritorio/ProyectoTFG/results/RESULTSSTACKING.txt


In [10]:
def load_results(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        df_list.append(df)
    results_df = pd.concat(df_list, ignore_index=True)
    drop_cols = [col for col in results_df.columns if any(char.isdigit() for char in col)]
    results_df.drop(columns=drop_cols, inplace=True)
    # Asegurar que NaN en la columna 'Model' no cause problemas
    results_df = results_df[results_df['Model'].str.contains('VotingClassifier', na=False)]
    return results_df

def find_best_by_metric(results_df, metric):
    results_grouped = results_df.groupby('Window')
    best_models = {}
    for window, group in results_grouped:
        max_value = group[metric].max()
        best_models[window] = group[group[metric] == max_value]
    return best_models

def create_stacking_report(folder_path, output_file_path):
    results_df = load_results(folder_path)
    
    best_by_accuracy = find_best_by_metric(results_df, 'Accuracy')
    best_by_bal_acc = find_best_by_metric(results_df, 'Sensibilidad Promedio')
    best_by_g_mean = find_best_by_metric(results_df, 'G-Mean')

    with open(output_file_path, 'w') as file:
        file.write("VotingClassifier Performance Report\n\n")
        
        windows = results_df['Window'].unique()
        for window in sorted(windows):
            file.write(f"Window: {window}\n")
            if window in best_by_accuracy:
                file.write("Best Accuracy:\n")
                file.write(best_by_accuracy[window].to_string(index=False))
                file.write("\n\n")
            if window in best_by_bal_acc:
                file.write("Best mean Recall:\n")
                file.write(best_by_bal_acc[window].to_string(index=False))
                file.write("\n\n")
            if window in best_by_g_mean:
                file.write("Best G-Mean:\n")
                file.write(best_by_g_mean[window].to_string(index=False))
                file.write("\n\n")
    
    print(f'Report has been saved to {output_file_path}')

# Asegúrate de definir 'output_path' correctamente
report_file_path = '/home/ximo/Escritorio/ProyectoTFG/results/RESULTSVOTING.txt'
create_stacking_report(output_path, report_file_path)


Report has been saved to /home/ximo/Escritorio/ProyectoTFG/results/RESULTSVOTING.txt


In [20]:
output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS'
fichero_con_resultados = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTS.csv'
report_file_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTS.txt'

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)
results_df.to_csv(fichero_con_resultados, index=False)  # Ajusta esta ruta según sea necesario

In [21]:
import pandas as pd

# Cargar el archivo CSV
file_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTS.csv'
df = pd.read_csv(file_path)

# Calcular la media de Accuracy, Sensibilidad Promedio y Specificidad General por modelo
grouped_df = df.groupby('Model').agg({
    'Accuracy': 'mean',
    'Sensibilidad Promedio': 'mean',
    'Especificidad General': 'mean'
}).reset_index()

# Ordenar los modelos por Accuracy medio en orden descendente
sorted_df = grouped_df.sort_values(by='Accuracy', ascending=False)

# Guardar los resultados en un archivo de texto
output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSmedios.txt'
with open(output_path, 'w') as f:
    f.write(f"{'Model':<15}{'Accuracy Medio':<15}{'Sensibilidad Promedio Media':<30}{'Specificidad General Media':<30}\n")
    f.write("="*90 + "\n")
    for _, row in sorted_df.iterrows():
        f.write(f"{row['Model']:<15}{row['Accuracy']:<15.4f}{row['Sensibilidad Promedio']:<30.4f}{row['Especificidad General']:<30.4f}\n")

print("Resultados guardados en RESULTSmedios.txt")


Resultados guardados en RESULTSmedios.txt


In [23]:
import pandas as pd
import os

output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS'
result_file_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTS.csv'
report_file_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/windows.txt'

def create_window_ranking(output_path, result_file):
    df = pd.read_csv(result_file)
    # Filter only the required models
    df = df[df['Model'].isin(['StackingClassifier', 'RandomForest', 'CatBoost', 'VotingClassifier'])]
    metrics = ['Accuracy', 'Sensibilidad Promedio', 'Especificidad General']

    results = {}
    for metric in metrics:
        # Calculate the sum of each metric per window and divide by the number of entries to get the average
        grouped = df.groupby('Window')[metric].agg(['sum', 'count'])
        grouped['average'] = grouped['sum'] / grouped['count']
        results[metric] = grouped['average'].sort_values(ascending=False)

    with open(report_file_path, 'w') as file:
        for metric, data in results.items():
            file.write(f"Best window for {metric}:\n")
            for window, score in data.items():
                # Change here to ensure all decimals are shown
                file.write(f"{window} {score}\n")
            file.write("\n")

# Execute the function to process and save the rankings
create_window_ranking(output_path, result_file_path)


In [14]:
def order_results_by_model_and_window(results_df):
    # Ordenar primero por modelo de forma alfabética y luego por ventana
    ordered_df = results_df.sort_values(by=['Model', 'Window'], ascending=[True, True])
    return ordered_df

def order_results_by_window_and_model(results_df):
    # Convertir Window a entero para un orden numérico correcto, si es necesario
    results_df['Window'] = results_df['Window'].astype(int)
    # Ordenar primero por ventana de forma ascendente y luego por modelo de forma alfabética
    ordered_df = results_df.sort_values(by=['Window', 'Model'], ascending=[True, True])
    return ordered_df

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)  # Asegúrate de que esta función se haya llamado correctamente anteriormente

# Ordenar y guardar los archivos
results_ordered_by_model = order_results_by_model_and_window(results_df)
results_ordered_by_window = order_results_by_window_and_model(results_df)

# Guardar los archivos CSV ordenados
results_ordered_by_model.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDMODEL.csv', index=False)
results_ordered_by_window.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDWINDOW.csv', index=False)

print("Los archivos ordenados han sido guardados.")


Los archivos ordenados han sido guardados.


In [15]:
def order_results_by_metric(results_df, metric):
    # Ordenar por la métrica especificada de forma descendente
    ordered_df = results_df.sort_values(by=[metric], ascending=False)
    return ordered_df

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)  

# Ordenar y guardar los archivos por cada métrica
results_ordered_by_accuracy = order_results_by_metric(results_df, 'Accuracy')
results_ordered_by_bal_acc = order_results_by_metric(results_df, 'Sensibilidad Promedio')
results_ordered_by_g_mean = order_results_by_metric(results_df, 'G-Mean')
results_ordered_by_specifity = order_results_by_metric(results_df, 'Especificidad General')


# Guardar los archivos CSV ordenados
results_ordered_by_accuracy.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDACCURACY.csv', index=False)
results_ordered_by_bal_acc.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDBALANCEDACCURACY.csv', index=False)
results_ordered_by_g_mean.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDGMEAN.csv', index=False)
results_ordered_by_specifity.to_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDESPECIFITY.csv', index=False)

print("Los archivos ordenados por métricas han sido guardados.")

Los archivos ordenados por métricas han sido guardados.


In [16]:
import pandas as pd

# Cargar los ficheros ordenados
accuracy_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDACCURACY.csv')
balanced_acc_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDBALANCEDACCURACY.csv')
g_mean_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDESPECIFITY.csv')

def calculate_ranking(*dataframes):
    model_scores = {}

    # Calcular puntuaciones para cada métrica
    for df in dataframes:
        for i, row in enumerate(df.itertuples(), start=1):
            if row.Model in model_scores:
                model_scores[row.Model].append(i)
            else:
                model_scores[row.Model] = [i]
    
    # Normalizar puntuaciones dividiendo por el número de apariciones
    for model, scores in model_scores.items():
        model_scores[model] = sum(scores) / len(scores)

    # Ordenar los modelos por su puntuación promedio
    ranked_models = sorted(model_scores.items(), key=lambda item: item[1])

    return ranked_models

# Calcular rankings
accuracy_ranking = calculate_ranking(accuracy_df)
balanced_accuracy_ranking = calculate_ranking(balanced_acc_df)
g_mean_ranking = calculate_ranking(g_mean_df)

# Escribir los resultados en un archivo de texto
with open('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/analisis.txt', 'w') as file:
    file.write("Ordenados por Accuracy:\n")
    for model, score in accuracy_ranking:
        file.write(f"{model}: {score:.2f}\n")
    file.write("\nOrdenados por mean Recall:\n")
    for model, score in balanced_accuracy_ranking:
        file.write(f"{model}: {score:.2f}\n")
    file.write("\nOrdenados por Specifity:\n")
    for model, score in g_mean_ranking:
        file.write(f"{model}: {score:.2f}\n")

print("El análisis de ranking ha sido guardado en analisis.txt.")


El análisis de ranking ha sido guardado en analisis.txt.


In [19]:
import pandas as pd

def calculate_weighted_ranking(df, metric):
    # Asegurar que el DataFrame está ordenado por la métrica
    df = df.sort_values(by=[metric], ascending=False)
    # Agregar una columna de ranking basado en la posición
    df['Rank'] = df[metric].rank(method='average', ascending=False)
    # Agrupar por modelo y calcular el promedio de los rangos
    df_grouped = df.groupby('Model')['Rank'].mean().reset_index()
    df_grouped = df_grouped.sort_values('Rank')
    return df_grouped

def compile_all_rankings(accuracy_df, balanced_acc_df, g_mean_df):
    # Calcular los rankings ponderados para cada métrica
    accuracy_ranking = calculate_weighted_ranking(accuracy_df, 'Accuracy')
    balanced_accuracy_ranking = calculate_weighted_ranking(balanced_acc_df, 'Sensibilidad Promedio')
    g_mean_ranking = calculate_weighted_ranking(g_mean_df, 'Especificidad General')
    
    # Preparar un DataFrame general para el ranking combinado
    combined = accuracy_ranking.rename(columns={'Rank': 'Rank_Accuracy'})
    combined = combined.merge(balanced_accuracy_ranking.rename(columns={'Rank': 'Rank_Balanced_Accuracy'}), on='Model', how='outer')
    combined = combined.merge(g_mean_ranking.rename(columns={'Rank': 'Rank_Especifidad'}), on='Model', how='outer')
    
    # Calcular el ranking promedio
    combined['Average_Rank'] = combined[['Rank_Accuracy', 'Rank_Balanced_Accuracy', 'Rank_Especifidad']].mean(axis=1)
    combined = combined.sort_values('Average_Rank')

    # Escribir todos los rankings en un solo archivo
    with open('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/ranking.txt', 'w') as f:
        f.write("Ranking by Accuracy:\n")
        f.write(accuracy_ranking.to_string(index=False) + "\n\n")
        f.write("Ranking by mean Recall:\n")
        f.write(balanced_accuracy_ranking.to_string(index=False) + "\n\n")
        f.write("Ranking by Especifidad:\n")
        f.write(g_mean_ranking.to_string(index=False) + "\n\n")
        f.write("General Ranking Across Metrics:\n")
        f.write(combined[['Model', 'Average_Rank']].to_string(index=False))

    print("Los rankings han sido compilados y guardados en 'ranking.txt'.")

# Cargar los datos
accuracy_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDACCURACY.csv')
balanced_acc_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDBALANCEDACCURACY.csv')
g_mean_df = pd.read_csv('/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTSORDEREDESPECIFITY.csv')

# Compilar los rankings y generar el archivo
compile_all_rankings(accuracy_df, balanced_acc_df, g_mean_df)

Los rankings han sido compilados y guardados en 'ranking.txt'.


In [18]:
import pandas as pd
import os

# Paths
result_file_path = '/home/ximo/Escritorio/ProyectoTFG/resultsIGUALADOS/RESULTS.csv'

def print_top_accuracy_rows(result_file, window, top_n=3):
    # Load data from the results file
    df = pd.read_csv(result_file)
    
    # Ensure that 'Window' is interpreted as an integer, if not already
    df['Window'] = df['Window'].astype(int)

    # Filter for the specified window
    window_df = df[df['Window'] == window]
    
    # Check if there are any entries for this window
    if not window_df.empty:
        # Sort by 'Accuracy' and get the top N rows, considering ties
        top_accuracy_rows = window_df.nlargest(top_n, 'Accuracy')
        print(f"Top {top_n} rows with the highest accuracy for window {window}:")
        print(top_accuracy_rows)
    else:
        print(f"No data found for window {window}.")

# Specify the window you are interested in
window_of_interest = 4
print_top_accuracy_rows(result_file_path, window_of_interest)


Top 3 rows with the highest accuracy for window 4:
                 Model  Window   K  Accuracy    G-Mean  F1-Score General  \
15            CatBoost       4  42  0.566667  0.558823          0.905413   
5     VotingClassifier       4  50  0.559375  0.548262          0.797867   
7   StackingClassifier       4  45  0.556250  0.546879          0.974050   

    Sensibilidad Promedio  Especificidad General  F1-Score_Música  \
15               0.566667               0.855556         0.958722   
5                0.559375               0.853125         0.879658   
7                0.556250               0.852083         0.989691   

    Sensitivity_Música  ...  F1-Score_Leer  Sensitivity_Leer  \
15            0.566667  ...       0.882629          0.559722   
5             0.630556  ...       0.716438          0.533333   
7             0.540278  ...       0.975818          0.498611   

    Specificity_Leer  F1-Score_Meditar  Sensitivity_Meditar  \
15          0.829630          0.852528         

In [24]:
output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsEXTRA'
fichero_con_resultados = '/home/ximo/Escritorio/ProyectoTFG/resultsEXTRA/RESULTS.csv'


# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)
results_df.to_csv(fichero_con_resultados, index=False)  # Ajusta esta ruta según sea necesario

In [30]:
output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsBINMATES'
fichero_con_resultados = '/home/ximo/Escritorio/ProyectoTFG/resultsBINMATES/RESULTS.csv'

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)
results_df.to_csv(fichero_con_resultados, index=False)  # Ajusta esta ruta según sea necesario

In [32]:
output_path = '/home/ximo/Escritorio/ProyectoTFG/resultsBINMUSICA'
fichero_con_resultados = '/home/ximo/Escritorio/ProyectoTFG/resultsBINMUSICA/RESULTS.csv'

# Llamar a la función y guardar los resultados
results_df = summarize_results(output_path)
results_df.to_csv(fichero_con_resultados, index=False)  # Ajusta esta ruta según sea necesario